In [72]:
import requests
import json
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
AUTHENTICATION = config['WeaG']['AUTHENTICATION']
StationName = '興海'

url1 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001'
url2 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001'
url3 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0002-001'

params = {'Authorization':AUTHENTICATION,
         'StationName':StationName}

r = requests.get(url1, params=params)
print(r.text)

{"success":"true","result":{"resource_id":"O-A0003-001","fields":[{"id":"StationName","type":"String"},{"id":"StationId","type":"String"},{"id":"CoordinateName","type":"String"},{"id":"CoordinateFormat","type":"String"},{"id":"StationAltitude","type":"String"},{"id":"IntScaleValue","type":"Float"},{"id":"StationAltitude","type":"Float"},{"id":"CountyName","type":"String"},{"id":"TownName","type":"String"},{"id":"CountyCode","type":"String"},{"id":"TownCode","type":"String"},{"id":"Weather","type":"String"},{"id":"VisibilityDescription","type":"String"},{"id":"SunshineDuration","type":"Float"},{"id":"Precipitation","type":"Float"},{"id":"WindDirection","type":"Float"},{"id":"WindSpeed","type":"Float"},{"id":"AirTemperature","type":"Float"},{"id":"RelativeHumidity","type":"Integer"},{"id":"AirPressure","type":"Float"},{"id":"UVIndex","type":"Float"},{"id":"PeakGustSpeed","type":"Float"},{"id":"DateTime","type":"DateTime"}]},"records":{"Station":[]}}


In [76]:
data = r.json()
if data['records']['Station']:

    S = data['records']['Station'][0]['StationName']
    O = data['records']['Station'][0]['ObsTime']['DateTime']
    T = data['records']['Station'][0]['WeatherElement']['AirTemperature']
    H = data['records']['Station'][0]['WeatherElement']['RelativeHumidity']/100
    print(S,O,T,H)
else:
    print('没有数据')


没有数据
